In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Define Rastrigin Function


In [12]:
def rastrigin(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

# Initiate population

In [13]:
def generate_particle(num_variables,swarm_size,x_min,x_max):
    swarm = np.random.uniform(x_min,x_max,(swarm_size,num_variables))

    velocity = np.zeros_like(swarm)

    return swarm , velocity

In [14]:

def index_finder(arr):
    mini = 0
    for i in range(len(arr)):
        if rastrigin(arr)[i] < rastrigin(arr)[mini]:
            mini = i
    return mini 

# Particle Swarm Optimisation

In [15]:
def pso(num_iterations,swarm_size,num_variables,x_min,x_max,W,C1,C2,E):
    swarm,velocity = generate_particle(num_variables,swarm_size,x_min,x_max)
    local_best = np.copy(swarm)
    global_best = np.full_like(local_best,local_best[index_finder(swarm)])
    
    for z in range(num_iterations):
        swarm2 = swarm + (E * velocity)
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        ras_swarm = np.array(rastrigin(swarm))
        ras_swarm2 = np.array(rastrigin(swarm2))
        
        for i in range(swarm_size):
            if ras_swarm2[i] < ras_swarm[i]:
                local_best[i] = swarm2[i]
                continue
            if min(ras_swarm2) < min(ras_swarm):
                global_best = np.full_like(local_best,local_best[index_finder(swarm2)])
                # print(global_best[0])
                continue
            
        velocity = ((W*(W-.4)*velocity*z)/num_iterations) + C1 * np.random.uniform(0,1) * (local_best - swarm) + C2 * np.random.uniform(0,1) * (global_best - swarm)
    
    return global_best[0]

In [16]:
def pso1(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon):
    
    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    
    for i in range(num_iterations):
        
        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2,x_min,x_max)
        
        dicx = {tuple(swarm[i]): rastrigin(swarm)[i] for i in range(swarm_size)}
        dicy = {tuple(swarm2[i]): rastrigin(swarm2)[i] for i in range(swarm_size)}

        if i == 0:
            
            local_best = swarm
            global_best = list(min(dicx, key=lambda k: dicx[k]))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=lambda k: dicy[k])) if np.argmin(dicy) < np.argmin(dicx) else list(min(dicx, key=lambda k: dicx[k]))      

        velocity = (alpha * velocity + np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm)) + np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        
    return global_best


In [17]:
def pso2(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon):
    def rastrigin(X):
        return 10 * len(X) + sum([(x ** 2 - 10 * np.cos(2 * np.pi * x)) for x in X])

    def generate_particle(num_variables, swarm_size, x_min, x_max):
        swarm = np.random.uniform(x_min, x_max, (swarm_size, num_variables))
        velocity = np.zeros_like(swarm)
        return swarm, velocity

    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    swarm = np.clip(swarm, x_min, x_max)
    swarm_positions = [swarm.tolist()]

    for i in range(num_iterations):
        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2, x_min, x_max)
        
        dicx = {tuple(swarm[i]): rastrigin(swarm[i]) for i in range(swarm_size)}
        dicy = {tuple(swarm2[i]): rastrigin(swarm2[i]) for i in range(swarm_size)}

        if i == 0:
            local_best = swarm
            global_best = list(min(dicx, key=dicx.get))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=dicy.get)) if min(dicy.values()) < min(dicx.values()) else list(min(dicx, key=dicx.get))

        velocity = (alpha * velocity + np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm)) + np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        swarm = swarm2
        swarm_positions.append(swarm.tolist())

    return global_best, swarm_positions

In [18]:
print(pso(100,50,5,-5.12,+5.12,.9,.2,.7,.2))

[ 2.07539876 -0.23749715  0.95518025 -2.93726341 -2.13348599]


In [31]:
print(type(pso1(100,50,5,-5.12,+5.12,.9,.2,.7,.2)))

<class 'list'>


In [30]:
globalbest,swarm_positions = pso2(100,50,5,-5.12,+5.12,.9,.2,.7,.2)
print(type(globalbest))

<class 'list'>
